<a href="https://colab.research.google.com/github/joshiayush/cardmatch/blob/main/notebooks/card_seek_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install langchain >/dev/null
!pip install langchain-community >/dev/null
!pip install langchain-openai >/dev/null
!pip install pypdf >/dev/null
!pip install chromadb >/dev/null

In [60]:
import os
import getpass

In [61]:
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LANGCHAIN_API_KEY:")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:")

LANGCHAIN_API_KEY:··········
OPENAI_API_KEY:··········


In [62]:
import pathlib
import re
from typing import Dict, Any

import numpy as np
import pandas as pd
from langchain.schema import Document, BaseRetriever
from langchain.document_loaders import (
    PyPDFLoader,
    TextLoader,
    UnstructuredWordDocumentLoader,
    UnstructuredEPubLoader,
    WebBaseLoader,
)
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.base import Chain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

In [63]:
_cc_sheets = ["sbi", "axis", "hdfc"]


def load_cc_info() -> Dict:
    cc_info = dict()
    for sheet in _cc_sheets:
        df = pd.read_excel("cc_urls.xlsx", sheet_name=sheet)
        cc_info.update(df.set_index("card_name").T.to_dict())
    return cc_info

In [64]:
credit_cards_info = load_cc_info()

<ipython-input-63-481e6df65adc>:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  cc_info.update(df.set_index("card_name").T.to_dict())
<ipython-input-63-481e6df65adc>:8: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  cc_info.update(df.set_index("card_name").T.to_dict())


In [65]:
class DocumentLoaderException(Exception):
    pass

In [66]:
class EPubReader(UnstructuredEPubLoader):
    def __init__(self, file_path: str | list[str], **kwargs: Any):
        super().__init__(file_path, **kwargs, mode="elements", strategy="fast")

In [67]:
class DocumentLoader(object):
    """Loads in a document with a supported extension."""

    supported_extensions = {
        ".pdf": PyPDFLoader,
        ".txt": TextLoader,
        ".epub": EPubReader,
        ".docx": UnstructuredWordDocumentLoader,
        ".doc": UnstructuredWordDocumentLoader,
    }

In [68]:
def load_document(filepath: str) -> list[Document]:
  """Load a file and return it as a list of Document."""
  ext = pathlib.Path(filepath).suffix

  loader = DocumentLoader.supported_extensions.get(ext)
  if not loader:
    raise DocumentLoaderException(f"Unsupported file extension: {ext}")

  return loader(filepath).load()

In [69]:
def configure_retriever(docs: list[Document]) -> BaseRetriever:
    """Retriever to use."""
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    embeddings = OpenAIEmbeddings()
    vectordb = Chroma.from_documents(splits, embeddings)
    return vectordb.as_retriever(
        search_type="mmr", search_kwargs={"k": 2, "fetch_k": 4}
    )

In [70]:
def configure_chain(retriever: BaseRetriever) -> Chain:
    """Configure chain with a retriever."""
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
    return ConversationalRetrievalChain.from_llm(
        llm, retriever=retriever, memory=memory, verbose=True,
    )

In [71]:
def configure_qa_chain(cc_info: Dict) -> Chain:
    """Read documents, configure retriever, and the chain."""
    docs = []
    for key, info in cc_info.items():
        if info["card_link"] is np.nan:
            continue

        card_doc = WebBaseLoader(info["card_link"]).load()
        docs.extend(card_doc)

        tnc_doc = load_document(info["tnc"])
        docs.extend(tnc_doc)
        break

    retriever = configure_retriever(docs)
    return configure_chain(retriever)

In [72]:
qa_chain = configure_qa_chain(credit_cards_info)

In [73]:
qa_chain.run("What is the full name of the credit card?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Card App upon termination of your Mobile Credit Card 
  (b)  You are fully responsible for any disclosure of your SBI Credit Card Details, Device Passcode or other security details relating to your 
Eligible Device, Token Requestor and Mobile Credit Card to any other person, even if such disclosure is accidental or unauthorised. You are 
required to bear all risks and consequences of your Token Requestor and Mobile Credit Card being used by unauthorised persons or for 
unauthorised purposes. 
  (c)  You have to use an Eligible Device of a type or model specified by Token Requestor from time to time to register, store and use Mobile 
Credit Card in your Mobile Wallet. Token Requestor have the right to

'The full name of the credit card is "SBI Credit Card."'

In [74]:
qa_chain.run("What is the joining fee of this credit card?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the full name of the credit card?
Assistant: The full name of the credit card is "SBI Credit Card."
Follow Up Input: What is the joining fee of this credit card?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
SBI CARDS-MOST IMPORTANT TERMS & CONDITIONS
1. *FEES AND CHARGES
A. Annual Fees & Renewal Fees
There is Annual Fee and Renewal Fee applicable on the SBI Credit Card (SBI Card). Annual fee is a 
one-time charge ranging between Rs.0 to Rs.9,999 plus applicable taxes and r

'The joining fee, referred to as the annual fee, for the SBI Credit Card ranges between Rs.0 to Rs.9,999 plus applicable taxes. The specific fee applicable to a cardholder would be communicated at the time of applying for the credit card.'

In [75]:
qa_chain.run("What benefits do I get in movie tickets every month?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the full name of the credit card?
Assistant: The full name of the credit card is "SBI Credit Card."
Human: What is the joining fee of this credit card?
Assistant: The joining fee, referred to as the annual fee, for the SBI Credit Card ranges between Rs.0 to Rs.9,999 plus applicable taxes. The specific fee applicable to a cardholder would be communicated at the time of applying for the credit card.
Follow Up Input: What benefits do I get in movie tickets every month?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try 

'With the SBI Card ELITE variant, you can avail a discount on movie tickets every month. The total amount payable for the tickets would be reduced by Rs. 500 or the cost of 2 tickets, whichever is less. This offer is valid on primary cards only and is subject to the terms and conditions of BookMyShow.'